In [34]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import integrate
import geopy.distance
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import requests
import time
import math
import itertools
import sys


import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [35]:
# cred = credentials.Certificate('./../Backend/serviceAccount.json')
# app_firebase = firebase_admin.initialize_app(cred)
# db = firestore.client()

In [36]:
sys.path.append('./../Backend/')
from key_value.token import apiKey, apiUrl, auth_header

In [37]:
change_type = {
    # "A320"      : "737800",
    "A320"      : "727200",
    # "737-800"   : "737800",
    "737-800"   : "727200",
    "737-900"   : "737800"
}

# mockup_api = ['AIQ311-1670716320-schedule-0614','AIQ3102-1670715240-schedule-0556']
mockup_api = ['AIQ311-1670716320-schedule-0614']


#----- 50 -----#
# lat_start =  13.811475
# long_start = 100.505997
#----- 40 -----#
lat_start =  13.831475
long_start = 100.525997

L_dB_start = 0
size = 40

In [38]:
#----- read fix_point to get power setting -----#
df_fix_point = pd.read_csv('./../Preprocess/data/ANP2.2_Default_fixed_point_profiles.csv',delimiter=';', skiprows=[0])
# df_fix_point = pd.read_csv('./../Preprocess/data/ANP2.2_Default_fixed_point_profiles.csv',delimiter=';')

df_fix_point = df_fix_point[df_fix_point['Stage Length'] == 1]
df_fix_point = df_fix_point[df_fix_point['Op Type'] == "D"]

#----- read NPD to get L(dB) by [power setting - distance]-----#
npd = pd.read_csv('./../Preprocess/data/ANP2.2_NPD_data.csv',delimiter=';', skiprows=[0])

In [39]:
#----- calculate distance from lat, long -----#
def add_distance(lat1, long1, lat2, long2, h2):
    position1 = (lat1,long1)
    position2 = (lat2,long2)
    a = geopy.distance.geodesic(position1, position2).ft
    b = h2
    b = b*100
    c_square = a**2 + b**2
    c = math.sqrt(c_square)
    return c


In [40]:

#----- function Single Soung Event -----#
def function_L(sound) :
    return 10**(sound / 10)

def get_feet(distance):
    feet = distance.split('_')[1]
    return float(feet[:-2])

def npd_inter_per_flight(df_param, npd_id):
    npd_of_flight = npd[npd['NPD_ID'] == npd_id]
    npd_of_flight = npd_of_flight[npd_of_flight['Noise Metric'] == 'LAmax']
    npd_of_flight = npd_of_flight[npd_of_flight['Op Mode'] == 'D']
    npd_of_flight.set_index('Power Setting', inplace=True)

    npd_of_flight = npd_of_flight.reindex(npd_of_flight.index.union(df_param['Power Setting'].unique()))
    npd_of_flight = npd_of_flight.interpolate(method='index',limit_direction='both',limit=200)
    print(npd_of_flight)
    
    npd_of_flight = npd_of_flight.drop(columns=['NPD_ID','Noise Metric','Op Mode']).T.reset_index()
    npd_of_flight.rename(columns={'index':'distance'},inplace=True)

    npd_of_flight['distance'] = npd_of_flight['distance'].apply(get_feet)
    npd_of_flight.set_index('distance',inplace=True)

    return npd_of_flight

def add_powersetting_2_flight(df_param, df_fixpoint):


    # df__tmp = df_fixpoint.copy().groupby('TAS (kt)').max()
    df__tmp = df_fixpoint.copy()
    # display(df__tmp)

    df__tmp = df__tmp.groupby('TAS (kt)').max()
    # df__tmp = df__tmp.groupby('Altitude AFE (ft)').max()
    # display(df__tmp)
    # print("before",df__tmp.shape)
    df__tmp = df__tmp.reindex(df__tmp.index.union(df_param.groundspeed.unique()))
    # df__tmp = df__tmp.reindex(df__tmp.index.union(df_param.altitude.unique()))
    # print("after",df__tmp.shape)

    df__tmp = df__tmp.interpolate(method='index',limit_direction='both',limit=200)
    df__tmp = df__tmp.reindex(df_param.groundspeed.unique())
    # df__tmp = df__tmp.reindex(df_param.altitude.unique())
    df__tmp.reset_index(drop=False, inplace=True)
    df__tmp.rename(columns={'TAS (kt)':'groundspeed'}, inplace=True)
    # df__tmp.rename(columns={'Altitude AFE (ft)':'altitude'}, inplace=True)
    df__tmp = df__tmp[['groundspeed', 'Power Setting']]
    # df__tmp = df__tmp[['altitude', 'Power Setting']]

    # df_param = df_param.merge(df__tmp, how='left', on='altitude')
    df_param = df_param.merge(df__tmp, how='left', on='groundspeed')
    
    return df_param

def create_observerer():
    global lat_start, long_start, L_dB_start, size
    observer = np.zeros((size,size,3))
    for i in range(size) :
        for j in range(size) :
            observer[i][j] = lat_start, long_start, L_dB_start
            long_start += 0.004
        lat_start  += 0.004
        long_start = 100.525997
    lat_start =  13.831475
    return observer

In [41]:
def Calculate_Grid(observer,df_param, npd_param):
    # npd_after_inter = npd_param.copy()
    for i in range(len(observer)):
        for j in range(len(observer[i])): # j = [lat, long, L(dB)]
            #----- copy df_param -----#
            tmp = df_param.copy()
            
            #----- calculate distance every transection -----#
            tmp_distance = []
            tmp['distance'] = -1
            
            for index, row in tmp.iterrows():
                distance = add_distance(observer[i][j][0], observer[i][j][1], row['latitude'], row['longitude'], row['altitude'])
                # print(distance)
                tmp_distance.append(distance)
                tmp.loc[index, 'distance'] = distance
            
            #----- calculate L_dB from powersetting & distance -> copy to model_data -----#
            npd_after_inter = npd_param.reindex(npd_param.index.union(tmp_distance))
            # npd_after_inter = npd_after_inter.reindex(npd_after_inter.index.union(tmp_distance))
            npd_after_inter = npd_after_inter.interpolate(method='index',limit_direction='both',limit=200)
            model_data = npd_after_inter.copy()
            # display(model_data)
            
            #----- add L_dB to df['sound'] & select -----#
            for index, row in tmp.iterrows():
                tmp.loc[index,'sound'] = model_data.loc[row.distance,row['Power Setting']]
                # print(tmp.loc[index,'sound'])
                if tmp.loc[index,'sound'] != tmp.loc[index,'sound']:
                    tmp.loc[index,'sound'] = 0

            
            #----- Single Sound Event Model -----#
            # if tmp.empty:
            #     continue
            tmp['Function'] = tmp['sound'].apply(function_L)
            area = integrate.simpson(tmp['Function'])
            #----- change -inf -> 0 -----#
            if area == 0:
                lSEL = 0
            else:
                lSEL = 10*np.log10( (1/1)*area)
                # if lSEL != lSEL:
                #     print('have NAN2')
                if lSEL <= 40 or lSEL >=85:
                    print("have ",lSEL)
        
            
            #----- add l_dB to observer -----#
            observer[i][j][2] = lSEL
        
    model_data.to_csv('tmp_model.csv')

    return observer

In [42]:
def generate_grid(flight, db):
    df = {}

    result_func = {
    'status':1,
    'id':flight
    }
    
    doc_ref = db.collection('detail_and_grid').document('detail').collection(f'{flight}').order_by("timestamp")
    docs = list(doc_ref.stream())
    flight_dict = list(map(lambda x: x.to_dict(), docs))
    df['df'] = pd.DataFrame(flight_dict)
    
    #----- set air_type, NPD_ID -----#
    aircraft = flight.split('-')[0]
    # print(aircraft)
    doc = db.collection('aircraft_engine').document(f'{aircraft}').get()
    if doc.exists:
        doc = doc.to_dict()
        air_type = doc['type']
        df['ACFT_ID'] = air_type
        # print(air_type)
        # doc2 = db.collection('NPD_ID').document(f'{air_type}').get()
        # if doc2.exists:
        #     doc2 = doc2.to_dict()
        #     df['NPD_ID'] = doc2['NPD_ID']
        # else: 
        #     print('not found NPD : ', flight, ' Aircraf_type : ', air_type)
        #     return False
    else: 
        # print('not found',aircraft)
        response = requests.get(apiUrl + f"flights/{aircraft}",headers=auth_header)

        if response.status_code == 200:
            # print("got aircraft_type")
            aircraft_type = response.json()['flights'][0]['aircraft_type']
            
            #----- get engine data from aircraft_type -----#
            response = requests.get(apiUrl + f"aircraft/types/{aircraft_type}",headers=auth_header)
            
            if response.status_code == 200:
                # print("got engine data")
                engine_data = response.json()
                engine_data.pop('manufacturer', None)
                engine_data.pop('description', None)
                engine_data['flight_id'] = aircraft
                
                #----- add to firestore -----#
                doc_ref = db.collection(f'aircraft_engine').document(f'{aircraft}')
                doc_ref.set(engine_data)
            else:
                result_func['status'] =0
                result_func['message'] = 'Error executing request'
                return result_func  
                
            df['ACFT_ID'] = aircraft

        else:
            result_func['message'] = 'Error executing request'
            result_func['status'] =0
            return result_func
            
    airtype = df['ACFT_ID']       
    doc2 = db.collection('NPD_ID').document(f'{airtype}').get()
    if doc2.exists:
        doc2 = doc2.to_dict()
        df['NPD_ID'] = doc2['NPD_ID']
    else: 
        # print('not found NPD : ', flight, f' Aircraf_type : \"{airtype}\"')
        result_func['status'] =0
        result_func['message'] = f'not found NPD : {flight}, Aircraf_type : "{airtype}"'
        return result_func
    NPDForCheck = df['NPD_ID']
        
    # print(f'airtype :{airtype}, npd: {NPDForCheck}, flight_ID: {flight}')
    result_func['message'] = f'airtype :{airtype}, npd: {NPDForCheck}, flight_ID: {flight}'
    # return result_func
    
    #----- set period -----#
    doc2 = db.collection('filter_flight').document(f'{flight}').get()
    if doc2.exists:
        doc2 = doc2.to_dict()
        period = doc2['period']
        df['period'] = period
    
    #----- add data_fix_point -----#
    aircraft_type = df["ACFT_ID"]
    if aircraft_type in change_type:
        aircraft_type = change_type[aircraft_type]

    df['df_fix_point'] = df_fix_point[df_fix_point.ACFT_ID == aircraft_type]
    df['df_fix_point'] = df['df_fix_point'].reset_index(drop=True)
    #----- add power setting to df -----#
    df['df'] = add_powersetting_2_flight(df['df'], df['df_fix_point'])
    
    # #########################
    # display(df['df'])
    # return df['df']
    # df['df'].to_csv('check_power.csv')

    #----- get NPD table -----#
    df['npd'] = npd_inter_per_flight(df['df'], df['NPD_ID'])
    
    #----- create grid [lat, long, 0] -----#
    df['grid'] = create_observerer()


    #----- get grid from calculate_Grid -----#
    df['grid'] = Calculate_Grid(df['grid'], df['df'], df['npd'])
    
    #----- TO PIVOT TABLE -----#
    df_tmp = pd.DataFrame(columns = ['Lat','Long','L_dB'])

    #----- matrix to pandas -----#
    for i in range(len(df['grid'])):
        row = pd.DataFrame(df['grid'][i], columns = ['Lat','Long','L_dB'])
        df_tmp = pd.concat([df_tmp, row])
    
    #----- Create pivot table -----#
    tabal_df = pd.pivot_table(df_tmp,index='Lat',columns='Long',values='L_dB')

    df['grid'] = tabal_df
    df['grid'] = df['grid'].iloc[::-1]
    
    result_func['df'] = df['grid']
    return result_func

In [43]:

# test =generate_grid(mockup_api, db)
test =generate_grid('AIQ3029-1670715240-schedule-0553', db)


               NPD_ID Noise Metric Op Mode  L_200ft  L_400ft  L_630ft  \
Power Setting                                                           
10712.0000        NaN          NaN     NaN      NaN      NaN      NaN   
10726.7875        NaN          NaN     NaN      NaN      NaN      NaN   
10741.5750        NaN          NaN     NaN      NaN      NaN      NaN   
10756.3625        NaN          NaN     NaN      NaN      NaN      NaN   
10771.1500        NaN          NaN     NaN      NaN      NaN      NaN   
10785.9375        NaN          NaN     NaN      NaN      NaN      NaN   
10800.7250        NaN          NaN     NaN      NaN      NaN      NaN   
10815.5125        NaN          NaN     NaN      NaN      NaN      NaN   
10830.3000        NaN          NaN     NaN      NaN      NaN      NaN   
10845.0875        NaN          NaN     NaN      NaN      NaN      NaN   
10859.8750        NaN          NaN     NaN      NaN      NaN      NaN   
10874.6625        NaN          NaN     NaN      NaN

KeyboardInterrupt: 

In [302]:
test

{'status': 1,
 'id': 'AIQ3029-1670715240-schedule-0553',
 'message': 'airtype :A320, npd: CFM565, flight_ID: AIQ3029-1670715240-schedule-0553',
 'df': Long       100.525997  100.529997  100.533997  100.537997  100.541997  \
 Lat                                                                     
 13.987475   56.100063   56.100063   56.100063   56.100063   56.100063   
 13.983475   56.100063   56.100063   56.100063   56.100063   56.100063   
 13.979475   56.100063   56.100063   56.100063   56.100063   56.100063   
 13.975475   56.100063   56.100063   56.100063   56.100063   56.100063   
 13.971475   56.100063   56.100063   56.100063   56.100063   56.100063   
 13.967475   56.100063   56.100063   56.100063   56.100063   56.100063   
 13.963475   56.100063   56.100063   56.100063   56.100063   56.100063   
 13.959475   56.100063   56.100063   56.100063   56.100063   56.100063   
 13.955475   56.100063   56.100063   56.100063   56.100063   56.100063   
 13.951475   56.536602   56.100063  

In [303]:
test['df']

Long,100.525997,100.529997,100.533997,100.537997,100.541997,100.545997,100.549997,100.553997,100.557997,100.561997,...,100.645997,100.649997,100.653997,100.657997,100.661997,100.665997,100.669997,100.673997,100.677997,100.681997
Lat,,,,,,,,,,,,,,,,,,,,,
13.987475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
13.983475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
13.979475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
13.975475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
13.971475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
13.967475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
13.963475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
13.959475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
13.955475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.166484,56.266072,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063


In [304]:
# test['df'].to_csv('change_to_d.csv')

In [305]:
# test.tail(30)

In [306]:
doc_ref2 = db.collection('detail_and_grid').document('grid').collection('AIQ3029-1670715240-schedule-0553')


In [307]:
test['df'].reset_index(inplace=True)

In [308]:
test['df']

Long,Lat,100.525997,100.52999700000001,100.53399700000001,100.53799700000002,100.54199700000002,100.54599700000003,100.54999700000003,100.55399700000004,100.55799700000004,...,100.64599700000015,100.64999700000016,100.65399700000016,100.65799700000017,100.66199700000017,100.66599700000017,100.66999700000018,100.67399700000018,100.67799700000019,100.6819970000002
0,13.987475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
1,13.983475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
2,13.979475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
3,13.975475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
4,13.971475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
5,13.967475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
6,13.963475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
7,13.959475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
8,13.955475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.166484,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
9,13.951475,56.536602,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.236161,56.348185,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063


In [309]:
# test['df'] = test['df'].astype(str)
test['df'].columns = [str(col) for col in test['df'].columns]
# [str(col) for col in test['df'].columns]
# test['df'] = test['df'].applymap(str)
    
# test['df'].isna().sum()
# for i in test['df'].isna().sum():
#     if i != 0:
#         print(i)
# test['df'].head()

In [310]:
# test['df'] = test['df'].dropna()


In [311]:
test['df'].head()

,Lat,100.525997,100.52999700000001,100.53399700000001,100.53799700000002,100.54199700000002,100.54599700000003,100.54999700000003,100.55399700000004,100.55799700000004,...,100.64599700000015,100.64999700000016,100.65399700000016,100.65799700000017,100.66199700000017,100.66599700000017,100.66999700000018,100.67399700000018,100.67799700000019,100.6819970000002
0,13.987475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
1,13.983475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
2,13.979475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
3,13.975475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063
4,13.971475,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,...,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063,56.100063


In [312]:
# postdata = test['df'].reset_index().to_dict('records')
postdata = test['df'].to_dict('records')
# list(map(lambda x: doc_ref2.add(x), postdata))
# postdata


[(DatetimeWithNanoseconds(2023, 2, 5, 9, 50, 53, 520255, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7ff1ba8c99a0>),
 (DatetimeWithNanoseconds(2023, 2, 5, 9, 50, 53, 597941, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7ff1ba37a5e0>),
 (DatetimeWithNanoseconds(2023, 2, 5, 9, 50, 53, 670038, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7ff1ba799070>),
 (DatetimeWithNanoseconds(2023, 2, 5, 9, 50, 53, 765235, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7ff198e6b6a0>),
 (DatetimeWithNanoseconds(2023, 2, 5, 9, 50, 53, 864577, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7ff199345df0>),
 (DatetimeWithNanoseconds(2023, 2, 5, 9, 50, 53, 960952, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7ff199550790>),
 (Datetime

In [313]:
list(map(lambda x: x, postdata))


[{'Lat': 13.987474999999982,
  '100.525997': 56.100063346890224,
  '100.52999700000001': 56.100063346890224,
  '100.53399700000001': 56.100063346890224,
  '100.53799700000002': 56.100063346890224,
  '100.54199700000002': 56.100063346890224,
  '100.54599700000003': 56.100063346890224,
  '100.54999700000003': 56.100063346890224,
  '100.55399700000004': 56.100063346890224,
  '100.55799700000004': 56.100063346890224,
  '100.56199700000005': 56.100063346890224,
  '100.56599700000005': 56.100063346890224,
  '100.56999700000006': 56.100063346890224,
  '100.57399700000006': 56.100063346890224,
  '100.57799700000007': 56.100063346890224,
  '100.58199700000007': 56.100063346890224,
  '100.58599700000008': 56.100063346890224,
  '100.58999700000008': 56.100063346890224,
  '100.59399700000009': 56.100063346890224,
  '100.59799700000009': 56.100063346890224,
  '100.6019970000001': 56.100063346890224,
  '100.6059970000001': 56.100063346890224,
  '100.6099970000001': 56.100063346890224,
  '100.6139970

In [314]:
# postdata
for row in postdata:
    # print(row)
    for i,j in row.items():
        # print(i, j)
        if not i or not j:
            print(i, j)
    # break

In [315]:
# postdata